# ECON408: Computational Methods in Macroeconomics

Geometric Series, Fixed Points, and Asset Pricing

Jesse Perla (University of British Columbia)

# Overview

## Motivation and Materials

-   In this lecture, we will introduce **fixed points**, practice a
    little Julia coding, move on to **geometric series**
-   The applications will be to **asset pricing** and **Keynesian
    multipliers**
    -   Asset pricing, in particular, will be something we come back to
        repeatedly as a way to practice our tools
-   Even for those not interested in finance, you will see that many
    problems are tightly related to asset pricing
    -   Human capital accumulation, choosing when to accept jobs, etc.

## Materials

-   Adapted from QuantEcon lectures coauthored with John Stachurski and
    Thomas J. Sargent
    -   [Julia by
        Example](https://julia.quantecon.org/getting_started_julia/julia_by_example.html)
    -   [Geometric Series for Elementary
        Economics](https://julia.quantecon.org/tools_and_techniques/geom_series.html)

In [1]:
using LinearAlgebra, Statistics, Plots, Random, Distributions, LaTeXStrings
default(;legendfontsize=16)

LoadError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.

# Intro to Fixed Points

## Fixed Points

-   Fixed points are everywhere!
    -   Lets first look at the mechanics and practice code, then apply
        them.
-   Take a mapping $f : X \to X$ for some set $X$.
    -   If there exists an $x^* \in X$ such that $f(x^*) = x^*$, then
        $x^*$: is called a “fixed point” of $f$
-   A fixed point is a property of a function, and may not be unique
-   Lets walk through the math, and then practice a little more Julia
    coding with them

## Simple, Linear Example

-   For given scalars $y,\beta$ and a scalar $v$ of interest

$$
v = y + \beta v
$$

-   If $|\beta| < 1$, then this can can be solved in closed form as
    $v = y/(1 - \beta)$
-   Rearrange the equation in terms of a map
    $f : \mathbb R \to \mathbb R$

$$
f(v) := y + \beta v
$$

-   Therefore, a fixed point $f(\cdot)$ is a solution to the above
    problem such that $v = f(v)$

## Value Function Iteration

-   Consider iteration of the map $f$ starting from an initial condition
    $v_0$

    $$
    v^{n+1} = f(v^n)
    $$

-   Does this converge? Depends on $f(\cdot)$, as we will explore in
    detail

    -   It shouldn’t depend on $v^0$ or there is an issue

-   See [Banach’s fixed-point
    theorem](https://en.wikipedia.org/wiki/Banach_fixed-point_theorem)

## When to Stop Iterating?

-   If $v^n$ is a scalar, then we can check convergence by looking at
    $|v^{n+1} - v^n|$ with some threshold, which may be problem
    dependent
    -   If $v^n$ will be a vector, so we should use a norm
        $||v^{n+1} - v^n||$
    -   e.g. the Euclidean norm, `norm(v_new - v_old)` in Julia
-   Keep numerical precision in mind! Can see this in Julia with the
    following

In [2]:
@show eps() #machine epsilon, the smallest number such that 1.0 + eps() > 1.0
@show 1.0 + eps()/2 > 1.0;

eps() = 2.220446049250313e-16
1.0 + eps() / 2 > 1.0 = false


## Verifying with the Linear Example

-   For our simple linear map: $f(v) \equiv y + \beta v$

-   Iteration becomes $v^{n+1} = y + \beta v^n$. Iterating backwards $$
    v^{n+1} = y + \beta v^n = y + \beta y + \beta^2 v^{n-1} = y \sum_{i=0}^{n-1} \beta^i + \beta^n v_0
    $$

    -   $\sum_{i=0}^{n-1} \beta^i = \frac{1 - \beta^n}{1 - \beta}$ and
        $\sum_{i=0}^{\infty} \beta^i = \frac{1}{1 - \beta}$ if
        $|\beta| < 1$
    -   So $n \to \infty$, converges to $v = y/(1 - \beta)$ for all
        $v_0$

## Implementing with For Loop

In [3]:
y = 1.0
beta = 0.9
v_iv = 0.8 # initial condition
v_old = v_iv
normdiff = Inf
iter = 1
for i in 1:1000
    v_new = y + beta * v_old # the f(v) map
    normdiff = norm(v_new - v_old)
    if normdiff < 1.0E-7 # check convergence
        iter = i
        break # converged, exit loop
    end
    v_old = v_new # replace and continue
end
println("Fixed point = $v_old  |f(x) - x| = $normdiff in $iter iterations");

Fixed point = 9.999999081896231  |f(x) - x| = 9.181037796679448e-8 in 154 iterations


## Implementing in Julia with While Loop

In [4]:
v_old = v_iv
normdiff = Inf
iter = 1
while normdiff > 1.0E-7 && iter <= 1000
    v_new = y + beta * v_old # the f(v) map
    normdiff = norm(v_new - v_old)
    v_old = v_new # replace and continue
    iter = iter + 1
end
println("Fixed point = $v_old  |f(x) - x| = $normdiff in $iter iterations")

Fixed point = 9.999999173706609  |f(x) - x| = 9.181037796679448e-8 in 155 iterations


## Avoid Global Variables

In [5]:
function v_fp(beta, y, v_iv; tolerance = 1.0E-7, maxiter=1000)
    v_old = v_iv
    normdiff = Inf
    iter = 1
    while normdiff > tolerance && iter <= maxiter
        v_new = y + beta * v_old # the f(v) map
        normdiff = norm(v_new - v_old)
        v_old = v_new
        iter = iter + 1
    end
    return (v_old, normdiff, iter) # returns a tuple
end
y = 1.0
beta = 0.9
v_star, normdiff, iter = v_fp(beta, y, 0.8)
println("Fixed point = $v_star |f(x) - x| = $normdiff in $iter iterations")

Fixed point = 9.999999173706609 |f(x) - x| = 9.181037796679448e-8 in 155 iterations


## Use a Higher Order Function and Named Tuple

-   Why hardcode the mapping? Pass it in as a function
-   Lets add in keyword arguments and use a named tuple for clarity

In [6]:
function fixedpointmap(f, iv; tolerance = 1.0E-7, maxiter=1000)
    x_old = iv
    normdiff = Inf
    iter = 1
    while normdiff > tolerance && iter <= maxiter
        x_new = f(x_old) # use the passed in map
        normdiff = norm(x_new - x_old)
        x_old = x_new
        iter = iter + 1
    end
    return (; value = x_old, normdiff, iter) # A named tuple
end

fixedpointmap (generic function with 1 method)

## Passing in a Function

In [7]:
y = 1.0
beta = 0.9
v_initial = 0.8
f(v) = y + beta * v # note that y and beta are used in the function!
sol = fixedpointmap(f, 0.8; tolerance = 1.0E-8) # don't need to pass
println("Fixed point = $(sol.value) |f(x) - x| = $(sol.normdiff) in $(sol.iter) iterations")

# Or use unpacking notation for the tuples, using anonymous function
(; value, normdiff, iter) = fixedpointmap(v -> y + beta * v, # creates an anonymous "closure"
                                          v_initial; tolerance = 1.0E-8)
println("Fixed point = $value |f(x) - x| = $normdiff in $iter iterations")

Fixed point = 9.999999918629035 |f(x) - x| = 9.041219328764782e-9 in 177 iterations
Fixed point = 9.999999918629035 |f(x) - x| = 9.041219328764782e-9 in 177 iterations


## Other Algorithms

-   VFI is instructive, but not always the fastest
-   Can also write as a “root finding” problem
    -   i.e. $\hat{f}(x) \equiv f(x) - x$ so that $\hat{f}(x^*) = 0$ is
        the fixed point
    -   These can be especially fast if $\nabla \hat{f}(\cdot)$ is
        available
-   Another is called Anderson Acceleration
    -   The fixed-point iteration we have above is a special case

## Use Packages with Better Algorithms

-   [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl/) has
    equations for solving equations (and fixed points)
    -   e.g., 3 iterations, not 177, for Andersen Acceleration
-   Uses multi-dimensional maps, so can write in that way rather than
    scalar

In [8]:
using NLsolve
# best style
y = 1.0
beta = 0.9
iv = [0.8] # note move to array
f(v) = y .+ beta * v # note that y and beta are used in the function!
sol = fixedpoint(f, iv) # uses Anderson Acceleration
fnorm = norm(f(sol.zero) .- sol.zero)
println("Fixed point = $(sol.zero) |f(x) - x| = $fnorm  in $(sol.iterations) iterations")

LoadError: ArgumentError: Package NLsolve not found in current path.
- Run `import Pkg; Pkg.add("NLsolve")` to install the NLsolve package.

# Geometric Series and PDVs

## Geometric Series

-   Finite geometric series

$$
1 + c + c^2 + c^3 + \cdots + c^T  = \frac{1 - c^{T+1}}{1-c}
$$

-   Infinite geometric series, requiring $|c| < 1$

$$
1 + c + c^2 + c^3 + \cdots = \frac{1}{1 -c }
$$

## Discounting

-   In discrete time, $t = 0, 1, 2, \ldots$
-   Let $r > 0$ be a one-period **net nominal interest rate**
-   A one-period **gross nominal interest rate** $R$ is defined as $$
    R = 1 + r > 1
    $$
-   If the nominal interest rate is $5$ percent, then $r= 0.05$ and
    $R = 1.05$

## Interpretation as Prices

-   The gross nominal interest rate $R$ is an **exchange rate** or
    **relative price** of dollars at between times $t$ and $t+1$. The
    units of $R$ are dollars at time $t+1$ per dollar at time $t$.
-   When people borrow and lend, they trade dollars now for dollars
    later or dollars later for dollars now.
-   The price at which these exchanges occur is the gross nominal
    interest rate.
    -   If I sell $x$ dollars to you today, you pay me $R x$ dollars
        tomorrow.
    -   This means that you borrowed $x$ dollars for me at a gross
        interest rate $R$ and a net interest rate $r$.
-   In equilibrium, the prices for borrowing and lending should be
    related

## Where do Interest Rates Come From?

-   More later, but consider connection to a discount factor
    $\beta \in (0,1)$ in **consumer preferences**
-   This represents how much consumers value future consumption tomorrow
    relative to today
-   In some simple cases $R^{-1} = \beta$ makes sense
    -   Much more later, including how to think about cases with
        randomness
-   For now, just use $R^{-1}$ directly as a discount factor, thinking
    about risk-neutrality

## Accumulation

-   $x, x R, x R^2, \cdots$ tells us how investment of $x$ dollar value
    of an investment **accumulate** through time. Compounding
-   Reinvested in the project (i.e., compounding)
    -   thus, $1$ dollar invested at time $0$ pays interest $r$ dollars
        after one period, so we have $r+1 = R$ dollars at time $1$
    -   at time $1$ we reinvest $1+r =R$ dollars and receive interest of
        $r R$ dollars at time $2$ plus the **principal** $R$ dollars, so
        we receive $r R + R = (1+r)R = R^2$ dollars at the end of period
        $2$

## Discounting

-   $1, R^{-1}, R^{-2}, \cdots$ tells us how to **discount** future
    dollars to get their values in terms of today’s dollars.
-   Tells us how much future dollars are worth in terms of today’s
    dollars.
-   Remember that the units of $R$ are dollars at $t+1$ per dollar at
    $t$.
    -   the units of $R^{-1}$ are dollars at $t$ per dollar at $t+1$
    -   the units of $R^{-2}$ are dollars at $t$ per dollar at $t+2$
    -   and so on; the units of $R^{-j}$ are dollars at $t$ per dollar
        at $t+j$

## Asset Pricing

-   An asset has payments stream of $y_t$ dollars at times
    $t = 0, 1, 2, \ldots, G \equiv 1+g, g > 0$ and $G < R \equiv 1 + r$

    $$
    y_t = G^t y_0
    $$

    -   i.e. grows at $g$ percent, discounted at $r$ percent

-   The **present value** of the asset is

    $$
    \begin{aligned} 
    p_0  & = y_0 + y_1/R + y_2/(R^2) + \cdots = \sum_{t=0}^{\infty} y_t (1/R)^t = \sum_{t=0}^{\infty} y_0 G^t (1/R)^t \\
    &= \sum_{t=0}^{\infty} y_0 (G/R)^t = y_0/(1 - G R^{-1})
    \end{aligned}
    $$

## Gordon Formula

-   For small $r$ and $g$, use a Taylor series or $r g \approx 0$ to get

    $$
    G R^{-1} \approx 1 + g - r
    $$

-   Hence,

$$
p_0  = y_0/(1 - (1+g)/(1+r)) \approx y_0 / (r - g)
$$

## Assets with Finite Lives

-   Consider an asset that pays $y_t = 0$ for $t > T$ and $y_t = y$ for
    $t \leq T$

-   The present value is

    $$
    \begin{aligned} 
    p_0  & \sum_{t=0}^{T} y_t (1/R)^t = \sum_{t=0}^{T} y_0 G^t (1/R)^t \\
    &= \sum_{t=0}^{T} y_0 (G/R)^t = y_0\frac{1 - (G/R)^{T+1}}{1 - G/R}
    \end{aligned}
    $$

-   How large is $(G/R)^{T+1}$?

    -   If small, then infinite horizon may be a good approximation

## Is Infinite Horizon a Reasonable Approximation?

-   Implement these in code to compare

In [9]:
infinite_payoffs(g, r, y_0) = y_0 / (1 - (1 + g) * (1 + r)^(-1))
function finite_payoffs(T, g, r, y_0)
    G = 1 + g
    R = 1 + r
    return (y_0 * (1 - G^(T + 1) * R^(-T - 1))) / (1 - G * R^(-1))
end
@show infinite_payoffs(0.01, 0.05, 1.0)
@show finite_payoffs(100, 0.01, 0.05, 1.0);

infinite_payoffs(0.01, 0.05, 1.0) = 26.24999999999994
finite_payoffs(100, 0.01, 0.05, 1.0) = 25.73063957477331


## Comparing Different Horizons

In [10]:
g = 0.01
r = 0.05
y_0 = 1.0
T = 100
# broadcast over 0:T
p_finite = finite_payoffs.(0:T, g, r, y_0)
p_infinite = infinite_payoffs(g, r, y_0)
plot(0:T, p_finite,xlabel = "T",
     label= L"p_0(T)", size = (600,400))
hline!([p_infinite], linestyle = :dash,
       label = L"p_0(\infty)")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[10]:9

## Discounting vs. Growth

-   For $T = \infty$, we assumed that $G R^{-1} < 1$, or approximately
    $g < r$

In [11]:
T = 10
y_0 = 1.0
plot(title = L"Present Value $p_0(T)$", legend = :topleft, xlabel = "T")
plot!(finite_payoffs.(0:T, 0.4, 0.9, y_0),
      label = L"r=0.9 \gg 0.4 = g")
plot!(finite_payoffs.(0:T, 0.4, 0.5, y_0), label = L"r=0.5 > 0.4 = g")
plot!(finite_payoffs.(0:T, 0.4, 0.4001, y_0),
      label = L"r=0.4001 \approx 0.4 = g")
plot!(finite_payoffs.(0:T, 0.5, 0.4, y_0), label = L"r=0.4 < 0.5 = g")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[11]:3

# Asset Pricing and Fixed Points

## Rewriting our Problem

-   Lets write a version of the model where $G = 1, y_0 = \bar{y}$, and
    relabel $\beta \equiv 1/R$

-   The asset price, $p_t$ starting at any $t$ is

    $$
    p_t = \sum_{j = 0}^{\infty}\beta^j \bar{y}
    $$

    -   We also know this is $p_t = \frac{\bar{y}}{1 - \beta}$ but lets
        temporarily forget that

-   Since this definition works at any $t$,

    $$
    \begin{aligned}
    p_t &= \bar{y} + \beta \bar{y} + \beta^2 \bar{y} + \cdots\\
    &= \bar{y} + \beta \left(\bar{y} + \beta\bar{y} + \cdots\right)\\
     &= \bar{y} + \beta p_{t+1}
    \end{aligned}
    $$

## Recursive Formulation

-   This shows that the PDV formula fulfills a recursive equation in
    $p_t$

$$
p_t = \bar{y} + \beta p_{t+1}
$$

-   We could also check that $p_t = \frac{\bar{y}}{1 - \beta}$ fulfills
    this equation
-   There are be other $p_t$ which fulfill it, but we won’t explore that
    here
-   In cases where the price is time-invariant, write this as a fixed
    point

$$
p = \bar{y} + \beta p \equiv f(p)
$$

## Recursive Interpretation

$$
p_t = y_t + \beta p_{t+1}
$$

-   The price $p_t$ is the sum of
    -   The payoffs you get that period
    -   The discounted price of how much you can sell it next period

## Solving Numerically

In [12]:
y_bar = 1.0
beta = 0.9
iv = [0.8]
f(p) = y_bar .+ beta * p
sol = fixedpoint(f, iv) # uses Anderson Acceleration
@show y_bar/(1 - beta), sol.zero;

LoadError: UndefVarError: `fixedpoint` not defined

## A More Complicated Example

-   Instead $\bar{y}$, asset may pay $y_L$ or $y_H$
    -   You don’t know the payoff $y_{t+1}$ until $t+1$ occurs
    -   You need to assign some probabilities of each occuring
-   As with the previous example, lets assume you hold onto the asset
    only a single period, then sell it
    -   Naturally, the value of the asset to both you and others depends
        on $y_{t+1}$
    -   We will see much more in [future
        lectures](https://julia.quantecon.org/multi_agent_models/markov_asset.html)

## Recursive Formulation

-   Assume two prices: $p_L$ and $p_H$ for the asset depending on the
    $y_t$ $$
    \begin{aligned}
    p_L &= y_L + \beta \left[ 0.5 p_L + 0.5 p_H \right]\\
    p_H &= y_H + \beta \left[ 0.5 p_L + 0.5 p_H \right]
    \end{aligned}
    $$

-   Stack $p \equiv \begin{bmatrix} p_L & p_H \end{bmatrix}^{\top}$ and
    $y \equiv \begin{bmatrix} y_L & y_H \end{bmatrix}^{\top}$

    $$
    p = y + \beta \begin{bmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{bmatrix} p
    $$

    -   We will see later how to write as a mathematical expectation

-   We could solve this as a linear equation, but lets use a fixed point

## Solving Numerically with a Fixed Point

In [13]:
y = [0.5, 1.5] #y_L, y_H
beta = 0.9
iv = [0.8, 0.8]
A = [0.5 0.5; 0.5 0.5]
sol = fixedpoint(p -> y .+ beta * A * p, iv) # f(p) := y + beta A p
p_L, p_H = sol.zero # can unpack a vector
@show p_L, p_H, sol.iterations
# p = y + beta A p =>  (I - beta A) p = y => p = (I - beta A)^{-1} y
@show (I - beta * A) \ y; # or $inv(I - beta * A) * y

LoadError: UndefVarError: `fixedpoint` not defined

# Keynesian Multipliers

## Model without Prices

-   $c$: **consumption**, $i$: **investment**, $g$: **government
    expenditures**, $y$ **national income**
-   Prices don’t adjust/exit to clear markets
    -   **Excess supply** of labor and capital (unemployment and unused
        capital)
    -   Prices and interest rates fail to adjust to make aggregate
        **supply equal demand** (e.g., prices and interest rates are
        frozen)
    -   National income entirely determined by aggregate demand,
        $\uparrow c\implies \uparrow y$

## Simple Model

-   **Assume**: consume a fixed fraction $0 < b < 1$ of the national
    income $y_t$
    -   $b$ is the **marginal propensity to consume (MPC)**
    -   $1-b$ is the **marginal propensity to save**
    -   Modern macro would have $b$ adjust to reflect prices, consumer
        preferences, etc. and add in prices/production functions
-   Leads to three equations in this basic model
    -   An accounting identity for the national income, the investment
        choice, and the consumer choice above

## Equations

-   **National income** is an accounting identity: the sum of
    consumption, investment, and government expenditures is the national
    income

    $$
    y_t = c_t + i_t + g_t
    $$

-   **Investment** is the sum of private investment and government
    investment. Assume it is fixed here at $i$ and $g$

-   **Consumption** $c_t = b y_{t-1}$, i.e. lag on last periods
    income/output

## Dynamics of Income and Consumption

-   Substituting the consumption equation into the national income
    equation

    $$
    \begin{aligned}
    y_t &= c_t + i + g\\
    y_t &= b y_{t-1} + i + g\\
    y_t &= b (b y_{t-2} + i + g) + i + g\\
    y_t &= b^2 y_{t-2} + b (i + g) + (i + g)
    \end{aligned}
    $$

-   Iterative backwards to a $y_0$,

$$
y_t = \sum_{j=0}^{t-1} b^j (i + g) + b^t y_0 = \frac{1 - b^{t}}{1 - b} (i + g) + b^t y_0
$$

## Keynesian Multiplier

-   Take limit as $t \to \infty$ to get

    $$
    \lim_{t\to\infty}y_t = \frac{1}{1 - b} (i + g)
    $$

-   Define the **Keynesian multiplier** is $1/(1-b)$

    -   More consumption delivers higher income, which delivers more
        consumption, compounding…
    -   $i \to i + \Delta$ implies $y \to y + \Delta/(1-b)$. Same with
        $g$

-   Is this correct (or useful) of a model?

    -   Probably not…gives intuition for more believable models
    -   Lets us practice difference equations

## Iterating the Difference Equations

$$
y_t = b y_{t-1} + i + g
$$

In [14]:
function calculate_y(i, b, g, T, y_0)
    y = zeros(T + 1)
    y[1] = i + b * y_0 + g
    for t in 2:(T + 1)
        y[t] = b * y[t - 1] + i + g
    end
    return y
end
y_limit(i, b, g) = (i + g) / (1 - b)

y_limit (generic function with 1 method)

## Plotting Dynamics

In [15]:
i_0 = 0.3
g_0 = 0.3
b = 2/3 # = MPC out of income
y_0 = 0
T = 100
plot(0: T,calculate_y(i_0, b, g_0, T, y_0);
     title = "Aggregate Output",
     size=(600,400), xlabel = L"t",
     label = L"y_t")
hline!([y_limit(i_0, b, g_0)];
       linestyle = :dash,
       label = L"y_{\infty}")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[15]:8

## MPCs

In [16]:
bs = round.([1 / 3, 2 / 3, 5 / 6, 0.9], digits = 2)
plt = plot(title = "Changing Consumption as a Fraction of Income",
           xlabel = L"t", ylabel = L"y_t", legend = :topleft)
[plot!(plt, 0:T, calculate_y(i_0, b, g_0, T, y_0), label = L"b = %$b")
 for b in bs]
plt

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[16]:3

## Governments Can (Magically) Expand Output

-   Remember the limitation is that demand is too low and there is
    excess supply of labor and/or capital
-   What if the government increases $g$ by $\Delta$?
    $y \to y + \Delta/(1-b)$
-   Assume we start at the $y_{\infty}$ for the $g=0.3$
    -   Then we simulate dynamics for a permanent change to $g_1 = 0.4$

## Plotting Dynamics for Government Intervention

In [17]:
y_lim = y_limit(i_0, b, g_0)
Delta_g = 0.1
y_1 = calculate_y(i_0, b,
                  g_0 + Delta_g,
                  T, y_lim) 
plot(0: T, y_1, title = "Aggregate Output",
     size=(600,400), xlabel = L"t",
     label = L"y_t")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[17]:7